In [17]:
import pandas as pd
import glob
import json
import re
import numpy as np
from IPython.display import display
set_fn = set()


In [18]:

def extract_config(line):
    line = line.rstrip(')\n')
    config = re.findall("(\w+)='?([^,']*)'?", line)                                                                                                                                                             
    return {k: v for k, v in config}             

def list_dict_reverse(L):
    """ Given a list of dict with identical keys, turn that into a dict of numpy arrays.
    """
    keys = L[0].keys()
    res = {}
    for k in keys:
        res[k] = np.asarray([l[k] for l in L])
    return res

def parse_log(fn):
    with open(fn, 'r') as f:
        cfg = extract_config(f.readline())
        train_logs, test_logs = [], []
        for line in f.readlines():
            try:
                parsed = json.loads(line)
            except json.JSONDecodeError:
                continue
            if parsed['mode'] == 'train':
                train_logs.append(parsed)
            elif parsed['mode'] == 'test':
                test_logs.append(parsed)
        train_logs = list_dict_reverse(train_logs)
        test_logs = list_dict_reverse(test_logs)
        return cfg, train_logs, test_logs
        
data = []
data_var = []
for fn in (glob.glob('../res_proto_1_adam/*') +
            glob.glob('../res_proto_2_adam/*')):
    print("Read", fn)
    if fn in set_fn:
        print("Already parsed", fn)
        continue
    set_fn.add(fn)
    try:
        cfg, tr, te = parse_log(fn)
    except:
        continue
    cfg.update({
        'final_length': te['length'][-1],
        'final_loss': te['loss_objs'][-1],
    })
    for k in te.keys():
        if k.startswith('length'):
            cfg.update({
                'init_' + k: te[k][0],
                'final_' + k: te[k][-1],
            })
    data_var.append(cfg)
df = pd.DataFrame(data_var)

Read ../res_proto_1_adam/142fcf07581a9459f21abf195f933ea7
Read ../res_proto_1_adam/9618e8245feb89bfd0e73dff270c3814
Read ../res_proto_1_adam/9cbd86b3c17143da6124d30f5f10db05_I
Read ../res_proto_1_adam/faf935b6feddaa27bf036e044ecefbbb_I
Read ../res_proto_1_adam/6769d503c18ebe0368f4a67034742502_I
Read ../res_proto_1_adam/e8544559fefced4d696acf8ab4b46c46
Read ../res_proto_1_adam/e6c4832633b6c2faec1cfdbc00681325
Read ../res_proto_1_adam/de26b589d6fe76cd9e3f0bb87a25a16b_I
Read ../res_proto_1_adam/959df9b0e90b362774dc63813eedbb1a_I
Read ../res_proto_1_adam/6769d503c18ebe0368f4a67034742502
Read ../res_proto_1_adam/4f8a3ae5727588bc063e46ae030f59ad
Read ../res_proto_1_adam/a73525cf78fea5cbee524b69d947469f
Read ../res_proto_1_adam/288bc39036389bdd2046da356de2a37f
Read ../res_proto_1_adam/590074bce1b184e6d94648127f33c755_I
Read ../res_proto_1_adam/092ca670df728ebfb5e54f2027555657
Read ../res_proto_1_adam/6d997e355f17f9d1ae02df734c5c791a
Read ../res_proto_1_adam/38e6f0d68cec738b9591168afea5d334
Re

In [6]:
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore', downcast='float'))


# Factors

## Temperature
High temperature seems higher accuracy, and lower final entropy for -1...

In [7]:
# fields = ['temperature', 'optimizer', 'lr', 'mode', 'random_seed', ]
fields = ['final_loss', 'final_length', 'lr', 'random_seed', 'optimizer', 'temperature']

criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
print(len(df.loc[criterion_1]))
a = df.loc[criterion_1][fields].groupby(['temperature'])
print(len(a))
print(a)
a.agg(['mean', 'std', 'count'])

104
4


final_loss                 final_length                        lr  \
                  mean       std count         mean       std count      mean   
temperature                                                                     
0.90         14.919643  7.173091    40     5.485052  2.125127    40  0.000295   
1.00         20.442610  9.856024    32     4.782994  2.878631    32  0.000353   
1.25         14.329491  9.542083    18     4.925134  2.024478    18  0.000444   
1.50         14.023396  8.859455    14     4.553142  2.332278    14  0.000471   

                            random_seed                  
                  std count        mean       std count  
temperature                                              
0.90         0.000183    40    1.925000  1.022503    40  
1.00         0.000217    32    1.093750  1.117583    32  
1.25         0.000309    18    1.277778  1.406102    18  
1.50         0.000354    14    1.500000  1.091928    14

## Optimisation

- Use 3e-4 or even less, maybe?
- It's hard to conclude about beta2

In [8]:

fields = ['final_loss', 'final_length', 'lr', 'adam_beta2', 'random_seed', 'optimizer', 'temperature', 'batch_size']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['optimizer', 'lr', 'adam_beta2'])
display(a.agg(['mean', 'std', 'count']))

a = df.loc[criterion_1][fields].groupby(['optimizer', 'lr', 'batch_size'])
display(a.agg(['mean', 'std', 'count']))

a = df.loc[criterion_1][fields].groupby(['optimizer', 'batch_size', 'adam_beta2'])
display(a.agg(['mean', 'std', 'count']))

final_loss                  final_length  \
                                  mean        std count         mean   
optimizer lr     adam_beta2                                            
adam      0.0001 0.900        9.201864   1.225841     7     5.368124   
                 0.990       10.170705   1.350236     6     5.582108   
          0.0003 0.900       13.857473   8.024948    25     5.382781   
                 0.990       16.165970   7.178251    26     5.325895   
                 0.999       17.824537   8.930562    27     5.082368   
          0.0010 0.900       16.826181  12.946056     2     3.500000   
                 0.990       31.213291   7.765552     6     3.500000   
                 0.999       22.164740  12.865035     5     3.103610   

                                            random_seed                  \
                                  std count        mean       std count   
optimizer lr     adam_beta2                                               
adam      0.0001 0.900       0.692190     7    1.714286  1.603567     7   
                 0.990       0.656555     6    2.333333  0.516398     6   
          0.0003 0.900       2.328108    25    0.960000  1.306395    25   
                 0.990       2.497325    26    1.692308  1.049542    26   
                 0.999       2.575581    27    1.444444  1.120897    27   
          0.0010 0.900       3.535534     2    2.500000  0.707107     2   
                 0.990       2.738613     6    1.666667  1.211060     6   
                 0.999       2.652476     5    1.600000  0.547723     5   

                            temperature                  batch_size  \
                                   mean       std count        mean   
optimizer lr     adam_beta2                                           
adam      0.0001 0.900         0.978571  0.128638     7   86.857140   
                 0.990         1.100000  0.309839     6   96.000000   
          0.0003 0.900         1.100000  0.214087    25  112.639999   
                 0.990         1.007692  0.205277    26  116.923080   
                 0.999         1.057407  0.154237    27   99.555557   
          0.0010 0.900         1.375000  0.176777     2   80.000000   
                 0.990         1.150000  0.223607     6   80.000000   
                 0.999         1.230000  0.277489     5  128.000000   

                                              
                                   std count  
optimizer lr     adam_beta2                   
adam      0.0001 0.900       51.314158     7  
                 0.990       49.574187     6  
          0.0003 0.900       35.919911    25  
                 0.990       31.278009    26  
                 0.999       44.670876    27  
          0.0010 0.900       67.882251     2  
                 0.990       52.581366     6  
                 0.999        0.000000     5

final_loss                  final_length  \
                                  mean        std count         mean   
optimizer lr     batch_size                                            
adam      0.0001 32.0         9.691207   1.992012     5     5.256551   
                 128.0        9.622655   0.868289     8     5.598346   
          0.0003 32.0        14.678879   7.095024    15     5.223351   
                 128.0       16.314785   8.400835    63     5.268515   
          0.0010 32.0        33.189716   5.690288     4     3.500000   
                 128.0       22.110771  11.704962     9     3.279783   

                                            adam_beta2                  \
                                  std count       mean       std count   
optimizer lr     batch_size                                              
adam      0.0001 32.0        0.492513     5   0.936000  0.049295     5   
                 128.0       0.741455     8   0.945000  0.048107     8   
          0.0003 32.0        1.758921    15   0.970800  0.044335    15   
                 128.0       2.591785    63   0.962714  0.044854    63   
          0.0010 32.0        2.886751     4   0.967500  0.045000     4   
                 128.0       2.585819     9   0.985000  0.032171     9   

                            random_seed                 temperature            \
                                   mean       std count        mean       std   
optimizer lr     batch_size                                                     
adam      0.0001 32.0          1.400000  1.341641     5    0.940000  0.054772   
                 128.0         2.375000  1.060660     8    1.093750  0.278308   
          0.0003 32.0          1.000000  1.069045    15    1.056667  0.198986   
                 128.0         1.460317  1.202489    63    1.053968  0.193712   
          0.0010 32.0          2.250000  1.500000     4    1.187500  0.239357   
                 128.0         1.555556  0.527046     9    1.227778  0.247628   

                                   
                            count  
optimizer lr     batch_size        
adam      0.0001 32.0           5  
                 128.0          8  
          0.0003 32.0          15  
                 128.0         63  
          0.0010 32.0           4  
                 128.0          9

final_loss                  final_length  \
                                      mean        std count         mean   
optimizer batch_size adam_beta2                                            
adam      32.0       0.900       15.686562   9.575799     8     3.973763   
                     0.990       20.837181  12.438532     8     5.061581   
                     0.999       13.651018   6.186235     8     5.793783   
          128.0      0.900       12.269606   6.804030    26     5.667550   
                     0.990       16.730726   8.106739    30     5.082442   
                     0.999       20.119919   9.991667    24     4.432988   

                                                       lr                  \
                                      std count      mean       std count   
optimizer batch_size adam_beta2                                             
adam      32.0       0.900       2.469403     8  0.000313  0.000295     8   
                     0.990       1.738558     8  0.000513  0.000412     8   
                     0.999       0.537295     8  0.000300  0.000000     8   
          128.0      0.900       1.916991    26  0.000296  0.000161    26   
                     0.990       2.576240    30  0.000343  0.000233    30   
                     0.999       2.978208    24  0.000446  0.000290    24   

                                random_seed                 temperature  \
                                       mean       std count        mean   
optimizer batch_size adam_beta2                                           
adam      32.0       0.900         0.875000  1.356203     8    1.068750   
                     0.990         1.875000  1.125992     8    0.981250   
                     0.999         1.125000  1.125992     8    1.112500   
          128.0      0.900         1.307692  1.407671    26    1.098077   
                     0.990         1.766667  1.006302    30    1.061667   
                     0.999         1.583333  1.017955    24    1.075000   

                                                 
                                      std count  
optimizer batch_size adam_beta2                  
adam      32.0       0.900       0.205178     8  
                     0.990       0.119336     8  
                     0.999       0.241646     8  
          128.0      0.900       0.217477    26  
                     0.990       0.246592    30  
                     0.999       0.166159    24

## Length costs

- hard to conclude!

In [9]:
fields = ['final_loss', 'final_length', 'lr', 'adam_beta2', 'random_seed',
          'optimizer', 'temperature', 'length_cost', 'ada_len_cost_thresh']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['ada_len_cost_thresh', 'length_cost'])
print(a)
a.agg(['mean', 'std', 'count'])

final_loss                  final_length  \
                                      mean        std count         mean   
ada_len_cost_thresh length_cost                                            
0.0                 0.00         18.506088  12.715582     3     6.000000   
                    0.01          9.236023   0.034591     3     6.000000   
                    0.03         34.619301   0.896024     2     1.000000   
                    0.10         21.557497  10.832428     8     3.225393   
                    0.30         26.410374   1.040740     2     1.000000   
                    1.00         32.072582   0.146931     6     1.000000   
5.0                 0.01         10.023862   0.496929     3     6.000000   
                    0.03         13.669459   7.134633     7     5.359721   
                    0.10          7.950337        NaN     1     5.992647   
                    0.30         13.666456   7.828872    15     5.593939   
                    1.00         15.981870   6.256439     2     6.659258   
10.0                0.01         15.566474   9.251262     6     5.143494   
                    0.03          9.345360   1.003552     4     5.970756   
                    0.10         13.903465   4.703430    11     7.032937   
                    0.30         13.608202   6.459644    15     5.209358   
15.0                0.00         39.482071        NaN     1     6.000000   
                    0.03         11.832818        NaN     1     6.000000   
                    0.10         19.752399  11.107979     4     6.250000   
                    0.30         11.737398   2.452022     6     5.864138   
                    1.00         18.941868   6.999175     4     2.970338   

                                                       lr                  \
                                      std count      mean       std count   
ada_len_cost_thresh length_cost                                             
0.0                 0.00         0.000000     3  0.000767  0.000404     3   
                    0.01         0.000000     3  0.000167  0.000115     3   
                    0.03         0.000000     2  0.001000  0.000000     2   
                    0.10         3.056415     8  0.000300  0.000000     8   
                    0.30         0.000000     2  0.000300  0.000000     2   
                    1.00         0.000000     6  0.000300  0.000000     6   
5.0                 0.01         0.000000     3  0.000233  0.000115     3   
                    0.03         1.694019     7  0.000600  0.000374     7   
                    0.10              NaN     1  0.000100       NaN     1   
                    0.30         2.234470    15  0.000340  0.000282    15   
                    1.00         1.896095     2  0.000300  0.000000     2   
10.0                0.01         2.030175     6  0.000533  0.000361     6   
                    0.03         0.052436     4  0.000300  0.000000     4   
                    0.10         1.116548    11  0.000264  0.000081    11   
                    0.30         2.172332    15  0.000260  0.000083    15   
15.0                0.00              NaN     1  0.001000       NaN     1   
                    0.03              NaN     1  0.000300       NaN     1   
                    0.10         3.500000     4  0.000475  0.000350     4   
                    0.30         1.242018     6  0.000300  0.000000     6   
                    1.00         0.681872     4  0.000300  0.000000     4   

                                adam_beta2                 random_seed  \
                                      mean       std count        mean   
ada_len_cost_thresh length_cost                                          
0.0                 0.00          0.960000  0.051962     3    1.333333   
                    0.01          0.960000  0.051962     3    1.333333   
                    0.03          0.994500  0.006364     2    0.500000   
                    0.10          0.974250  0.045828     8    0.250

# Best runs

In [12]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss', 
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size', 'vocab_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir', 'receiver_nlayers', 'sender_nlayers',
         'dropout', 'random_seed']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 14.0)
criterion_3 = (df['final_length'] < 5)

# all runs shld be like that, but I prefer to verify, in case I load the wrong runs
# criterion_6 = (df['receiver_mlp'] == 'True')  
# criterion_7 = (df['sender_squash_output'] >= '0.0')
# criterion_8 = (df['sender_mlp'] == 'True') 
# criterion_9 = (df['batch_size'] == '128') 
criterion_10 = (df['entropy_coef'] == 0.0) 


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3 & criterion_10]
a = sub_df[fields].groupby(['ada_len_cost_thresh', 'length_cost',     
                            'lr', 'vocab_size', 'temperature']) 
print(a)
a.agg(['mean', 'std', 'count'])

final_length  \
                                                                      mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature                
0.0                 0.1         0.0003 64.0       1.25            4.441176   
5.0                 0.3         0.0001 64.0       0.90            4.746324   
                                0.0003 64.0       1.25            4.268048   
10.0                0.3         0.0001 64.0       0.90            4.397059   
                                0.0003 64.0       1.25            3.868316   
15.0                0.3         0.0003 512.0      1.00            4.467246   
                    1.0         0.0003 512.0      0.90            3.443850   

                                                                               \
                                                                    std count   
ada_len_cost_thresh length_cost lr     vocab_size temperature                   
0.0                 0.1         0.0003 64.0       1.25              NaN     1   
5.0                 0.3         0.0001 64.0       0.90         0.244368     2   
                                0.0003 64.0       1.25              NaN     1   
10.0                0.3         0.0001 64.0       0.90         0.000000     2   
                                0.0003 64.0       1.25         0.077517     2   
15.0                0.3         0.0003 512.0      1.00              NaN     1   
                    1.0         0.0003 512.0      0.90              NaN     1   

                                                              final_loss  \
                                                                    mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature              
0.0                 0.1         0.0003 64.0       1.25          9.838125   
5.0                 0.3         0.0001 64.0       0.90         11.654680   
                                0.0003 64.0       1.25          8.299747   
10.0                0.3         0.0001 64.0       0.90         10.627255   
                                0.0003 64.0       1.25          9.737232   
15.0                0.3         0.0003 512.0      1.00         10.200574   
                    1.0         0.0003 512.0      0.90         13.369365   

                                                                               \
                                                                    std count   
ada_len_cost_thresh length_cost lr     vocab_size temperature                   
0.0                 0.1         0.0003 64.0       1.25              NaN     1   
5.0                 0.3         0.0001 64.0       0.90         1.545899     2   
                                0.0003 64.0       1.25              NaN     1   
10.0                0.3         0.0001 64.0       0.90         0.000000     2   
                                0.0003 64.0       1.25         0.058396     2   
15.0                0.3         0.0003 512.0      1.00              NaN     1   
                    1.0         0.0003 512.0      0.90              NaN     1   

                                                              final_length_1.0  \
                                                                          mean   
ada_len_cost_thresh length_cost lr     vocab_size temperature                    
0.0                 0.1         0.0003 64.0       1.25                3.700422   
5.0                 0.3         0.0001 64.0       0.90                4.374473   
                                0.0003 64.0       1.25                3.738397   
10.0                0.3         0.0001 64.0       0.90                4.062236   
                                0.0003 64.0       1.25                3.308017   
15.0                0.3         0.0003 512.0      1.00                4.229958   
                    1.0         0.0003 512.0      0.90                2.398734   

                                                                             

## Find a specific run

In [14]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss',
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 14.0)
criterion_3 = (df['final_length'] < 5)


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3]
print(sub_df)
print(sub_df['checkpoint_dir'].values)

    ada_H_cost_thresh  ada_len_cost_thresh  adam_beta2  batch_size  \
10                0.0                 15.0       0.999        32.0   
18                0.0                  5.0       0.900       128.0   
29                0.0                  0.0       0.900       128.0   
51                0.0                 15.0       0.990       128.0   
74                0.0                  5.0       0.990        32.0   
77                0.0                 10.0       0.900       128.0   
82                0.0                  5.0       0.990        32.0   
85                0.0                 10.0       0.900       128.0   
88                0.0                 10.0       0.900       128.0   
99                0.0                 10.0       0.900       128.0   

                                       checkpoint_dir  checkpoint_freq  cuda  \
10                 de26b589d6fe76cd9e3f0bb87a25a16b_I              0.0  True   
18  /Tmp/slurm.933434.0/bosctom/proto_1_adam_sd1/1...              0.